In [1]:
import numpy as np, matplotlib.pyplot as plt, matplotlib as mpl
import pandas as pd
%matplotlib inline

In [2]:
import instrument as ins, isochrones as iso
import galaxy as gal, driver 
import fit_model, utils, gpu_utils

pycuda not installed.
GPU acceleration not available, sorry


ImportError: No module named emcee

In [4]:
filters = [ins.Filter.HST_F475W(1.), ins.Filter.HST_F814W(1.)]
n_filters = len(filters)
iso_model = iso.Isochrone_Model(filters)
driver_gpu = driver.Driver(iso_model, gpu=True)

In [ ]:
n_scale = 1024

In [5]:
model_ssp = gal.Galaxy_SSP(np.array([0., -2., 2., 10.]))
model_full = gal.Galaxy_Model(np.array([0., -2., 0., 0., 0., 0., 0., 0., 0.]))

In [8]:
%%time
_ = driver_gpu.simulate(model_full, 1024)

CPU times: user 1.08 s, sys: 221 ms, total: 1.31 s
Wall time: 1.43 s


In [10]:
xbins, ybins = np.arange(-1.5, 4.6, 0.05), np.arange(-12, 15.6, 0.05)
bins = np.array([xbins, ybins])

In [ ]:
loglikes = {}
N_samples = 100
for N_scale in [128, 256, 512, 1024, 2048]:
    for log_Npix in [0., 2., 4.]:
        print((N_scale, log_Npix))
        temp = []
        model = gal.Galaxy_SSP(np.array([0., -2., log_Npix, 10.]))
        _, mags, _, _ = driver_gpu.simulate(model, N_scale)
        pcmd = utils.make_pcmd(mags)
        driver_gpu.initialize_data(pcmd, bins)
        assert(np.isclose(driver_gpu.loglike(pcmd), 0.))
        for i in range(N_samples):
            _, mags, _, _ = driver_gpu.simulate(model, N_scale)
            pcmd = utils.make_pcmd(mags)
            temp.append(driver_gpu.loglike(pcmd))
        loglikes[(N_scale, log_Npix)] = temp

(128, 0.0)
(128, 2.0)
(128, 4.0)
(256, 0.0)
(256, 2.0)
(256, 4.0)
(512, 0.0)
(512, 2.0)
(512, 4.0)
(1024, 0.0)
(1024, 2.0)


In [13]:
loglikes

{(128, 0.0): [-2048.021398271268, -3171.1733690008805],
 (128, 1.0): [-1172.2244478202274, -1147.2343711103479],
 (128, 2.0): [-1050.4976797815953, -895.50505036689515],
 (128, 3.0): [-396.03947588973915, -411.26483769873829],
 (128, 4.0): [-298.62420272438317, -260.95401798031781],
 (128, 5.0): [-51.95282863550878, -66.713937883128779],
 (256, 0.0): [-3938.5373822634792, -3573.5421068666647],
 (256, 1.0): [-2268.5777780797785, -2936.979287893731],
 (256, 2.0): [-1369.1592576601356, -1577.3729029416049],
 (256, 3.0): [-557.62268983308024, -575.06370197281115],
 (256, 4.0): [-243.1855822854827, -254.25519302629272],
 (256, 5.0): [-30.965254868361409, -50.581456904419852],
 (512, 0.0): [-7639.3732316654086, -6552.1138924109055],
 (512, 1.0): [-4011.5324220857128, -6507.9216570425679],
 (512, 2.0): [-2075.0868934427026, -1916.6019631581769],
 (512, 3.0): [-652.35232971917276, -626.27028133999329],
 (512, 4.0): [-144.77781000793226, -168.61180458249237],
 (512, 5.0): [-49.464072069868763, 